In [3]:
import os
import pathlib
import numpy as np
import cv2
from tensorflow import keras
from keras import layers, preprocessing, callbacks, losses, utils
import sys
import PyQt5
from PyQt5.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout, QHBoxLayout, QGridLayout,QFrame
from PyQt5.QtGui import QPixmap, QPainter, QPen, QColor
from PyQt5.QtCore import Qt
import time

In [ ]:
data_dir = pathlib.Path('/Users/valentindaveau/reseau/datasets/dataset/base_donnes_iccp_reduite')
img_height = 600
img_width = 400
batch_size = 3      #nombre d'images envoyées à la fois
epochs = 3  

input_shape=(img_height, img_width,3)
num_classes=17


In [5]:
train_data = utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=(img_height, img_width),
        shuffle=True,                                   #Intialement sans cette ligne : mélange les images pour éviter que le réseau ne catégorise par ordre d'arivée
        batch_size=batch_size,
    )

Found 1893 files belonging to 52 classes.
Using 1515 files for training.
Metal device set to: Apple M2

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



In [6]:
val_data = utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

Found 1893 files belonging to 52 classes.
Using 378 files for validation.


In [7]:
class_names = sorted([dir for dir in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, dir))])

card_dir = {
        '1_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/1_carreaux.png',
        '2_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/2_carreaux.png',
        '3_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/3_carreaux.png',
        '4_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/4_carreaux.png',
        '5_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/5_carreaux.png',
        '6_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/6_carreaux.png',
        '7_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/7_carreaux.png',
        '8_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/8_carreaux.png',
        '9_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/9_carreaux.png',
        '10_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/10_carreaux.png',
        '11_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/11_carreaux.png',
        '12_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/12_carreaux.png',
        '13_carreaux': '/Users/valentindaveau/reseau/datasets/dataset/carreaux/13_carreaux.png',
        '1_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/1_coeur.png',
        '2_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/2_coeur.png',
        '3_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/3_coeur.png',
        '4_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/4_coeur.png',
        '5_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/5_coeur.png',
        '6_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/6_coeur.png',
        '7_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/7_coeur.png',
        '8_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/8_coeur.png',
        '9_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/9_coeur.png',
        '10_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/10_coeur.png',
        '11_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/11_coeur.png',
        '12_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/12_coeur.png',
        '13_coeur': '/Users/valentindaveau/reseau/datasets/dataset/coeur/13_coeur.png',
        '1_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/1_pique.png',
        '2_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/2_pique.png',
        '3_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/3_pique.png',
        '4_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/4_pique.png',
        '5_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/5_pique.png',
        '6_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/6_pique.png',
        '7_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/7_pique.png',
        '8_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/8_pique.png',
        '9_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/9_pique.png',
        '10_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/10_pique.png',
        '11_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/11_pique.png',
        '12_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/12_pique.png',
        '13_pique': '/Users/valentindaveau/reseau/datasets/dataset/pique/13_pique.png',
        '1_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/1_trefle.png',
        '2_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/2_trefle.png',
        '3_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/3_trefle.png',
        '4_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/4_trefle.png',
        '5_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/5_trefle.png',
        '6_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/6_trefle.png',
        '7_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/7_trefle.png',
        '8_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/8_trefle.png',
        '9_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/9_trefle.png',
        '10_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/10_trefle.png',
        '11_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/11_trefle.png',
        '12_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/12_trefle.png',
        '13_trefle': '/Users/valentindaveau/reseau/datasets/dataset/trefle/13_trefle.png',
    }

In [8]:
model = keras.Sequential([
        layers.InputLayer(input_shape=input_shape),
        layers.Rescaling(1./255),
        layers.Conv2D(128, 4, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(64, 4, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(32, 4, activation='relu'),
        layers.MaxPooling2D(),
        layers.Conv2D(16, 4, activation='relu'),
        layers.MaxPooling2D(),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax',dtype='float32')   #utilise un système de probabilités : probabilité que l'image appartienne à chaque classe
    ])

In [9]:
model.compile(optimizer='adam',     #Descente de gradient stochastique
                  loss=losses.SparseCategoricalCrossentropy(from_logits=False),  #Essayer True
                  metrics=['accuracy'])

In [10]:
def predict_image(model, class_names, img_path, img_height, img_width):
    image_to_predict = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img_to_predict = np.expand_dims(cv2.resize(image_to_predict, (img_height, img_width)), axis=0)
    prediction = model.predict(img_to_predict)
    res = np.argmax(prediction, axis=1)[0]    #argmax renvoie la valeur max des probabilités calculées ie renvoie la classe la plus probable
    return class_names[res]

In [11]:
class ImageDisplay(QWidget):
    def __init__(self, img_a_predire, predictions):
        super().__init__()
        self.img_a_predire = img_a_predire
        self.predictions = predictions
        self.initUI()

    def initUI(self):
        # Layout principal en QGridLayout
        grid = QGridLayout()

        for idx, (img_path, prediction) in enumerate(zip(self.img_a_predire, self.predictions)):
            # Labels pour les images
            label_predicted = QLabel(self)
            label_predicted.setPixmap(QPixmap(img_path).scaled(200, 300, Qt.AspectRatioMode.KeepAspectRatio))
            label_predicted.setAlignment(Qt.AlignmentFlag.AlignCenter)

            label_to_predict = QLabel(self)
            label_to_predict.setPixmap(QPixmap(card_dir[prediction]).scaled(200, 300, Qt.AspectRatioMode.KeepAspectRatio))
            label_to_predict.setAlignment(Qt.AlignmentFlag.AlignCenter)

            # Captions
            caption_predicted = QLabel('Carte à prédire', self)
            caption_predicted.setAlignment(Qt.AlignmentFlag.AlignCenter)

            caption_to_predict = QLabel('Prédiction', self)
            caption_to_predict.setAlignment(Qt.AlignmentFlag.AlignCenter)

            # Placement dans la grille (2 colonnes et 2 lignes)
            row = idx % 2
            col = idx // 2

            grid.addWidget(label_predicted, row * 2, col * 2)
            grid.addWidget(caption_predicted, row * 2 + 1, col * 2)
            grid.addWidget(label_to_predict, row * 2, col * 2 + 1)
            grid.addWidget(caption_to_predict, row * 2 + 1, col * 2 + 1)

        self.setLayout(grid)
        self.setGeometry(100, 100, 800, 600)
        self.setWindowTitle('Image Prediction Display')

        # Set background color
        self.setAutoFillBackground(True)
        p = self.palette()
        p.setColor(self.backgroundRole(), QColor('#096a09'))
        self.setPalette(p)

In [12]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
print("GPUs détectés:", gpus)
if not gpus:
    raise RuntimeError("Aucun GPU détecté. Vérifie l'installation de tensorflow-metal.")

GPUs détectés: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
start_reseau = time.time()
start_pred = time.time()

tensorboard_callback = callbacks.TensorBoard(log_dir="logs", histogram_freq=1, write_images=True)
    
model.fit(
        train_data,
        validation_data=val_data,
        epochs=epochs,
        callbacks=[tensorboard_callback])

model.summary()

end_reseau = time.time()

t_reseau_s = end_reseau - start_reseau
t_reseau_h = t_reseau_s / 3600
print("Temps d'entrainement :", t_reseau_h, " heures")

Epoch 1/3
370/505 [====================>.........] - ETA: 27s - loss: 3.9349 - accuracy: 0.0396

In [18]:
start_pred=time.time()

img_a_predire = [
    '/Users/valentindaveau/reseau/prediction/Conversion 50 percent.png',
    '/Users/valentindaveau/reseau/prediction/Convertio IMG 5519.png',
    '/Users/valentindaveau/reseau/prediction/IMG 5521.png',
    '/Users/valentindaveau/reseau/prediction/IMG 5522.png'
]

def main():
    predictions = [predict_image(model, class_names, img, img_height, img_width) for img in img_a_predire]

    app = QApplication(sys.argv)
    ex = ImageDisplay(img_a_predire, predictions)
    ex.show()
    sys.exit(app.exec_())


end_pred=time.time()
t_pred_s = end_pred - start_pred
t_pred_h = t_pred_s / 3600
print("Temps de prédiction :", t_pred_h, " heures")

Temps de prédiction : 4.0001339382595486e-08  heures
